In [ ]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import shutil
import random
from tqdm import tqdm

In [ ]:
# 디렉토리 설정
source_root = r"C:\Users\USER\Desktop\47.물류공간 예측 데이터\3.개방데이터\1.데이터\Validation\01.원천데이터"
label_root  = r"C:\Users\USER\Desktop\47.물류공간 예측 데이터\3.개방데이터\1.데이터\Validation\02.라벨링데이터"

In [ ]:
# 1. 모든 이미지 파일 경로 인덱싱
print("🔍 이미지 파일 인덱싱 중...")
img_map = {}  # {파일명: 전체 경로}
for dirpath, _, filenames in os.walk(source_root):
    for fname in filenames:
        if fname.endswith('.jpg'):
            img_map[fname] = os.path.join(dirpath, fname)

print(f"✅ 총 이미지 파일 수: {len(img_map)}")

In [ ]:
# 2. 모든 JSON에서 이미지 이름 읽고 매칭
pairs = []

for dirpath, _, filenames in os.walk(label_root):
    for fname in filenames:
        if fname.endswith('.json'):
            json_path = os.path.join(dirpath, fname)
            try:
                with open(json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                file_name = data['images'][0]['file_name']  # <- 이미지 파일명

                if file_name in img_map:
                    pairs.append((img_map[file_name], json_path))
                else:
                    print(f"[❌] 이미지 없음: {file_name}")
            except Exception as e:
                print(f"[⚠️] JSON 처리 오류: {json_path} / {e}")

print(f"\n✅ 최종 매칭된 이미지-라벨 쌍: {len(pairs)}개")

In [ ]:
# ✅ YOLO 형식 저장 경로
yolo_root = r'C:\Users\USER\Desktop\dataset_yoloseg' # 데이터셋 저장 경로(디렉토리) 지정
image_dir = os.path.join(yolo_root, 'images')
label_dir = os.path.join(yolo_root, 'labels')

In [ ]:
# ✅ 비율 정의
random.seed(42)
random.shuffle(pairs)

N = len(pairs)
train_split = int(0.8 * N)
val_split = int(0.9 * N)

train_pairs = pairs[:train_split]
val_pairs = pairs[train_split:val_split]
test_pairs = pairs[val_split:]

splits = {'train': train_pairs, 'val': val_pairs, 'test': test_pairs}

In [ ]:
# ✅ YOLOv8 Seg 포맷 변환 함수
def coco_to_yolo_segmentation(annotation, image_w, image_h):
    class_id = annotation['category_id'] - 1  # YOLO class는 0부터 시작
    seg = annotation['segmentation'][0]
    norm_seg = [f"{x / image_w:.6f}" if i % 2 == 0 else f"{x / image_h:.6f}" for i, x in enumerate(seg)]
    return f"{class_id} " + " ".join(norm_seg)

In [ ]:
# ✅ 저장 루프
for split_name, split_pairs in splits.items():
    print(f"📁 {split_name.upper()} 세트 저장 중... ({len(split_pairs)}개)")
    img_out_dir = os.path.join(image_dir, split_name)
    lbl_out_dir = os.path.join(label_dir, split_name)
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(lbl_out_dir, exist_ok=True)

    for img_path, json_path in tqdm(split_pairs):
        try:
            # 이미지 복사
            fname = os.path.basename(img_path)
            shutil.copy(img_path, os.path.join(img_out_dir, fname))

            # JSON → YOLO Seg
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            img_info = data['images'][0]
            ann = data['annotations'][0]
            w, h = img_info['width'], img_info['height']
            yolo_line = coco_to_yolo_segmentation(ann, w, h)

            # 저장
            base = os.path.splitext(fname)[0]
            with open(os.path.join(lbl_out_dir, base + '.txt'), 'w') as f:
                f.write(yolo_line + '\n')

        except Exception as e:
            print(f"❌ 오류: {img_path}, {json_path} / {e}")

print("✅ 완료되었습니다!")